# Gemma2-9b
- GPU 큰것 필요 ex. A100

In [ ]:
import os
from google.colab import userdata

os.environ["KAGGLE_USERNAME"] = userdata.get('user_name')
os.envrion["KAGGLE_Key"] = userdata.get('kaggle')

os.envrion["KERAS_BACKEND"] = "jax" # Or "tensorflow" or "torch"

In [ ]:
!pip install -U keras-nlp
!pip install -U keras==3.33

In [ ]:
import keras_nlp
import keras

# 연산수행
keras.config.set_floatx("bfloat16")

gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_9b_en")
gemma_lm.summary()

In [ ]:
result = gemma_lm.generate("It was a dark and stormy night.", max_length=256)
print(result)

# Gemma_lora

In [ ]:
import os
from google.colab import userdata, drive

os.environ["KAGGLE_USERNAME"] = userdata.get("user_name")
os.environ["KAGGLE_KEY"] = userdata.get("kaggle")
os.environ["KERAS_BACKEND"] = "jax"  # Or "tensorflow" or "torch".
drive.mount("/content/drive")

In [ ]:
!pip install -q -U keras-nlp datasets
!pip install -q -U keras

os.environ["KERAS_BACKEND"] = "tensorflow"  # "jax" Or "torch" or "tensorflow".

# 트레이닝 설정
lora_name = "translator"
lora_rank = 8
lr_value = 1e-4

In [ ]:
import keras_nlp
import torch

tokenizer = keras_nlp.models.GemmaTokenizer.from_preset("gemma_instruct_2b_en")

from datasets import load_dataset

# 한글 데이터셋
ds = load_dataset("nlpai-lab/kullm-v2", split="train")

print(ds)
data = ds.with_format(
    "np", columns=["instruction", "output"], output_all_columns=False
)
train = []
max_token_len = 0

for x in data:
    item = f"user\n{x['instruction']}\nmodel\n{x['output']}"
    train.append(item)
    length = tokenizer(item).get_shape().as_list()[0]
    if length > max_token_len:
        max_token_len = length
        print(f"longest toekn {max_token_len} - {item}")

print(train[0])
print(train[1])
print(train[2])

print(max_token_len)

## Load Model

In [ ]:
import keras
import keras_nlp
import time

gemma = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")
gemma.summary()

In [ ]:
import keras
import keras_nlp

import time

gemma = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")
gemma.summary()

tick_start = 0

# 가지고 오는데 얼마나 걸리는지
def tick():
    global tick_start
    tick_start = time.time()

def tock():
    print(f"TOTAL TIME ELAPSED: {time.time() - tick_start:.2f}s")


def text_gen(prompt):
    tick()
    input = f"user\n{prompt}\nmodel\n"
    output = gemma.generate(input, max_length=512)
    print("\nGemma output:")
    print(output)
    tock()


text_gen("비언어적 커뮤니케이션에 대해 알려주세요")
#비언어적 커뮤니케이션도 언어적 커뮤니케이션만큼이나 중요합니다. 몸짓, 표정, 목소리 톤은 모두 메시지가 인식되는 방식에 중요한 역할을 합니다. 비언어적 단서가 언어적 메시지와 일치하는지 확인하세요. 예를 들어, 눈을 마주치고 열린 자세를 유지하는 것은 자신감과 진정성을 전달할 수 있습니다. 마찬가지로, 다른 사람의 비언어적 단서에 유의하세요. 비언어적 단서는 화자가 진정으로 무엇을 느끼고 생각하는지에 대한 귀중한 통찰력을 제공할 수 있습니다.

text_gen("존슨앤드존슨 COVID-19 백신")
#2020년 12월 11일 - 미국에서 화이자/바이오엔텍 COVID-19 백신이 긴급 사용 승인을 받았습니다.

# LoRA Fine-tuning


In [ ]:
# Enable LoRA for the model and set the LoRA rank (4, 8 or 16).
gemma.backbone.enable_lora(rank=lora_rank)
gemma.summary()

# Limit the input sequence length to 128 (to control memory usage).
gemma.preprocessor.sequence_length = 2048
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=lr_value,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

## LoRA epoch 저장
- 시간이 너무 오래 걸림. # 1266:40:17 분 걸림
- 아래 대체방법으로 Huggingface이용

In [ ]:
# 실제 실습시 삭제
# train_tmp = train[:5]

# Reduce the sequence length to a more manageable size.
gemma.preprocessor.sequence_length = 512  # Or an even smaller value if needed

for x in range(2):
    gemma.fit(train_tmp, epochs=1, batch_size=1)
    model_name = f"/content/drive/MyDrive/gemma_lora/{lora_name}_{lora_rank}_epoch{x+1}.lora.h5"
    gemma.backbone.save_lora_weights(model_name)

    text_gen("비언어적 커뮤니케이션에 대해 알려주세요")
    #비언어적 커뮤니케이션도 언어적 커뮤니케이션만큼이나 중요합니다. 몸짓, 표정, 목소리 톤은 모두 메시지가 인식되는 방식에 중요한 역할을 합니다. 비언어적 단서가 언어적 메시지와 일치하는지 확인하세요. 예를 들어, 눈을 마주치고 열린 자세를 유지하는 것은 자신감과 진정성을 전달할 수 있습니다. 마찬가지로, 다른 사람의 비언어적 단서에 유의하세요. 비언어적 단서는 화자가 진정으로 무엇을 느끼고 생각하는지에 대한 귀중한 통찰력을 제공할 수 있습니다.

    text_gen("존슨앤드존슨 COVID-19 백신")
    #2020년 12월 11일 - 미국에서 화이자/바이오엔텍 COVID-19 백신이 긴급 사용 승인을 받았습니다.

## 위의 방법 대체 - 시간 단축
- SFTTrainer 방식

In [ ]:
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
from huggingface_hub import login
from google.colab import userdata

login(
    token=userdata.get('huggingface'),
    add_to_git_credential=True
)

In [ ]:
# Dataset 준비
## 네이버 기사 요약 데이터셋
from datasets import load_dataset
#dataset = load_dataset("daekeun-ml/naver-news-summarization-ko")
dataset = load_dataset("wonik-hi/korea_summary_Thesis")

## dataset은 train과 train, validation, test로 구분되어 있음
dataset['train'][0]

In [ ]:
## Gemma 모델의 한국어 요약 테스트

### 모델로드
BASE_MODEL = "google/gemma-2b-it"
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)
doc = dataset['train']['instruction'][0]
print(doc)

In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

messages = [
    {
        "role": "user",
        "content": "다음 내용을 자연스럽고 문맥에 맞게 완성하세요:\n\n{}".format(doc)
     }
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
prompt

In [ ]:
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True
)

print(outputs[0]['generated_text'][len(prompt):])

In [ ]:
## Gemma-it 파인튜닝
def generate_prompt(example):
    output_texts = []
    for i in range(len(example['insturction'])):
        messages = [
            {
            "role": "user",
             "content": "다음 내용을 자연스럽고 문맥에 맞게 완성하세요:\n\n {}".format(example['insturction'][i])
             },
             {
                 "role": "assistant",
                 "content": "{}".format(example['output'][i])
                 }
        ]
        chat_message = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        output_texts.append(chat_message)

    return output_texts

In [ ]:
def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['instruction'])):
        prompt_list.append(r"""<bos><start_of_turn>user
                           다은 매용을 자연스럽고 문맥에 맞게 완성하고, 글을 요약해주세요:
                           
                           {}<end_of_turn>
                           <start_of_turn>model
                           {}<end_of_turn><eos>
                           """.format(example['instruction'][i], example['output'][i]))
    return prompt_list

In [ ]:
train_data = dataset['train']
print(generate_prompt(train_data[:1])[0])

In [ ]:
lora_config = LoraConfig(
    r=6,
    lora_alpha=8,
    lora_dropout=0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
BASE_MODEL="google/gemma-2b-it"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map="auto", quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)
tokenizer.padding_side = 'right'

In [ ]:
# trainer 실행
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
        max_steps=1000,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_8bit",
        warmup_steps=0.03,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_cofnig=lora_config,
    formatting_func=generate_prompt
)

In [ ]:
trainer.train()

In [ ]:
ADAPTER_MODEL = "lora_adapter"

trainer.model.save_pretrained(ADAPTER_MODEL)

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

model = model.merge_and_unload()
model.save_pretrained('gemma-2b-it-ko')

In [ ]:
# 한국어 요약 모델 추론
BASE_MODEL = "google/gemma-2b-it"
FINETUNE_MODEL = "./gemma-2b-it-ko"

finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={"":0})
tokneizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)

In [ ]:
pipe_finetuned = pipeline("text-generation", model=finetune_model, tokenizer=tokenizer, max_new_tokens=512)

In [ ]:
messages = [
    {
        "role": "user",
        "content": "다음 내용을 자연스럽고 문맥에 맞게 완성하고, 글을 요약해주세요.\n\n{}".format(doc)
    }
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

In [ ]:
outputs = pipe_finetuned(
    prompt,
    do_sample=True,
    temperature=0.2,
    top_k=50,
    top_p=0.95,
    add_special_tokens=True,
)
print(outputs[0]['generated_text'][len(prompt):])

In [ ]:
model.push_to_hub("wonik-hi/gemma-2b-it-ko")

In [ ]:
!export LC_ALL=en_US.UFT-8

In [ ]:
!pip install -q -U langchain

In [ ]:
!nvidia-smi

In [ ]:
!pip install -q streamlit

In [ ]:
!pip install diffusers

In [ ]:
%%writefile app.py

import streamlit as st
import torch
from diffusers import StableDiffusionPipeline
from PIL import Image
from torch import autocast

MODEL = "./gemma-2b-it-ko"

st.header("Streamlit Application")
pipe = StableDiffusionPipeline.from_pretrained(MODEL, revision="fp16", torch_dtype=torch.float16, use_auto_token=True)
pipe = pipe.to("cuda")

with st.form('form', clear_on_submit = True):
    prompt = st.text_input("Prompt: ", "")
    submitted = st.form_submit_button('Generate')

if submitted and prompt:
    prompt = [prompt]
    
    print(outputs[0]["generated_text"][len(prompt)])

In [ ]:
!streamlit run /content/app.py &>/content/logs.txt &
!npx localtunnel --port 8501